コード: https://github.com/yuki-inaho/dbscan_comparison

コミット: forkしたリポジトリのhotfix/memory_leaks_42e1e71ブランチの最新コミット(5bd612ab937889d83e5ef6f6714f843fcf4e2b46)

In [5]:
import os
import psutil

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

In [6]:
# DBSCAN実行後のメモリ使用量を取得
after_memory = get_memory_usage()
print(f"Memory usage after DBSCAN: {after_memory / 1024 / 1024:.2f} MB")


Memory usage after DBSCAN: 225.13 MB


In [7]:
import numpy as np
from dbscan import DBSCAN

# 初期のメモリ使用量を取得
initial_memory = get_memory_usage()
print(f"Initial memory usage: {initial_memory / 1024 / 1024:.2f} MB")

points = np.random.rand(100000, 3)
pcd_np = np.asarray(points)

labels_another, core_samples_mask = DBSCAN(pcd_np, eps=0.025, min_samples=10)

# DBSCAN実行後のメモリ使用量を取得
after_memory = get_memory_usage()
print(f"Memory usage after DBSCAN: {after_memory / 1024 / 1024:.2f} MB")

# メモリ使用量の差を計算
memory_diff = after_memory - initial_memory
print(f"Memory difference: {memory_diff / 1024 / 1024:.2f} MB")

Initial memory usage: 225.13 MB
Memory usage after DBSCAN: 225.65 MB
Memory difference: 0.52 MB
Memory usage after DBSCAN: 225.65 MB
Memory difference: 0.52 MB


In [8]:
# 複数回DBSCANを実行し、メモリ使用量を確認
num_iterations = 100

for i in range(num_iterations):
    points = np.random.rand(100000, 3)
    pcd_np = np.asarray(points)
    labels, core_samples = DBSCAN(pcd_np, eps=0.025, min_samples=10)
    current_memory = get_memory_usage()
    print(f"Iteration {i+1}: Memory usage: {current_memory / 1024 / 1024:.2f} MB")

Iteration 1: Memory usage: 226.14 MB
Iteration 2: Memory usage: 226.14 MB
Iteration 3: Memory usage: 226.89 MB
Iteration 4: Memory usage: 226.88 MB
Iteration 5: Memory usage: 226.89 MB
Iteration 6: Memory usage: 226.89 MB
Iteration 7: Memory usage: 226.89 MB
Iteration 8: Memory usage: 226.89 MB
Iteration 9: Memory usage: 226.88 MB
Iteration 10: Memory usage: 226.89 MB
Iteration 11: Memory usage: 226.89 MB
Iteration 12: Memory usage: 226.89 MB
Iteration 13: Memory usage: 226.89 MB
Iteration 14: Memory usage: 226.88 MB
Iteration 15: Memory usage: 226.89 MB
Iteration 16: Memory usage: 226.88 MB
Iteration 17: Memory usage: 226.89 MB
Iteration 18: Memory usage: 226.89 MB
Iteration 19: Memory usage: 226.88 MB
Iteration 20: Memory usage: 226.89 MB
Iteration 21: Memory usage: 226.89 MB
Iteration 22: Memory usage: 226.88 MB
Iteration 23: Memory usage: 226.89 MB
Iteration 24: Memory usage: 226.89 MB
Iteration 25: Memory usage: 226.89 MB
Iteration 26: Memory usage: 226.89 MB
Iteration 27: Memory 

In [9]:
import os
import psutil
import numpy as np
from dbscan import DBSCAN
import sys
import weakref
import gc

# 初期のメモリ使用量を取得
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

initial_memory = get_memory_usage()

# ランダムな点群データを生成
pcd_np = np.random.rand(100000, 3)

# DBSCANを実行
labels_another, core_samples_mask = DBSCAN(pcd_np, eps=0.025, min_samples=10)

# メモリ使用量と参照カウントを確認
print(f"Initial memory usage: {initial_memory / 1024 / 1024:.2f} MB")

# 初期参照カウントを確認
initial_ref_count_labels = sys.getrefcount(labels_another)
initial_ref_count_core_samples = sys.getrefcount(core_samples_mask)
print(f"Initial reference count of labels_another: {initial_ref_count_labels}")
print(f"Initial reference count of core_samples_mask: {initial_ref_count_core_samples}")

# 弱参照を作成
weak_labels = weakref.ref(labels_another)
weak_core_samples = weakref.ref(core_samples_mask)

# labels_another, core_samples_maskの参照を削除 + ガベージコレクションを実行
del labels_another, core_samples_mask
gc.collect()

# 参照カウントを再度取得
if weak_labels() is not None:
    post_gc_ref_count_labels = sys.getrefcount(weak_labels())
    print(f"Post-GC reference count of labels_another: {post_gc_ref_count_labels}")
else:
    print(f"Post-GC reference count of labels_another: None (object collected)")

if weak_core_samples() is not None:
    post_gc_ref_count_core_samples = sys.getrefcount(weak_core_samples())
    print(f"Post-GC reference count of core_samples_mask: {post_gc_ref_count_core_samples}")
else:
    print(f"Post-GC reference count of core_samples_mask: None (object collected)")

# メモリ使用量の差を計算
current_memory = get_memory_usage()
memory_diff = current_memory - initial_memory
print(f"Memory difference after GC: {memory_diff / 1024 / 1024:.2f} MB")


Initial memory usage: 227.68 MB
Initial reference count of labels_another: 2
Initial reference count of core_samples_mask: 2
Post-GC reference count of labels_another: None (object collected)
Post-GC reference count of core_samples_mask: None (object collected)
Memory difference after GC: 0.00 MB
